In [5]:
!pip install crewai crewai-tools openai python-dotenv

  Using cached jsonref-1.1.0-py3-none-any.whl.metadata (2.7 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached protobuf-5.29.5-cp38-abi3-macosx_10_9_universal2.whl.metadata (592 bytes)
  Using cached docker-7.1.0-py3-none-any.whl.metadata (3.8 kB)
INFO: pip is looking at multiple versions of instructor to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opentelemetry-exporter-otlp-proto-grpc to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 666.8/666.8 kB 17.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 723.4/723.4 kB 37.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 13.6 MB/s  0:00:01 eta 0:00:01
Using cached jsonref-1.1.0-py3-none-any.whl (9.4 kB)
Using cached protobuf-5.29.5-c

In [6]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_MODEL = os.getenv("OPENAI_MODEL", "gpt-4o-mini")

In [7]:
def load_canonical_instructions(path: str) -> str:
    with open(path, "r", encoding="utf-8") as f:
        return f.read()

CANONICAL_INSTRUCTIONS = load_canonical_instructions(
    "canonical_insurance_letter_instructions.txt"
)

In [8]:
from crewai import Agent

letter_generator_agent = Agent(
    role="Insurance Letter Generator",
    goal="Generate a complete insurance letter using canonical instructions",
    backstory=(
        "You are a senior insurance correspondence specialist. "
        "You strictly follow canonical insurance letter instructions "
        "and generate accurate, professional letters."
    ),
    llm=OPENAI_MODEL,
    verbose=True,
)

In [9]:
formatter_agent = Agent(
    role="Insurance Letter Formatter",
    goal="Ensure the insurance letter is properly formatted and structured",
    backstory=(
        "You are an expert in professional insurance document formatting. "
        "You enforce headers, spacing, sections, and consistency."
    ),
    llm=OPENAI_MODEL,
    verbose=True,
)


In [10]:
compliance_agent = Agent(
    role="Insurance Compliance Reviewer",
    goal="Ensure the letter meets insurance compliance and regulatory standards",
    backstory=(
        "You are an insurance compliance officer. "
        "You verify regulatory language, appeal rights, and legal disclaimers."
    ),
    llm=OPENAI_MODEL,
    verbose=True,
)


In [11]:
from crewai import Task

generate_letter_task = Task(
    description=(
        "Using the canonical instruction set below, generate an insurance letter.\n\n"
        "Canonical Instructions:\n"
        f"{CANONICAL_INSTRUCTIONS}\n\n"
        "Inputs:\n"
        "- Letter Type: {letter_type}\n"
        "- Company Name: {company_name}\n"
        "- Insured Name: {insured_name}\n"
        "- Policy Number: {policy_number}\n"
        "- Claim Number: {claim_number}\n\n"
        "Generate the full letter content."
    ),
    expected_output="A complete insurance letter draft",
    agent=letter_generator_agent,
)


In [12]:
format_letter_task = Task(
    description=(
        "Take the drafted insurance letter and ensure it is professionally formatted.\n"
        "Ensure:\n"
        "- Clear headers\n"
        "- Proper spacing\n"
        "- Section separators\n"
        "- Professional tone\n\n"
        "Do NOT remove any content."
    ),
    expected_output="A formatted insurance letter",
    agent=formatter_agent,
)

In [13]:
compliance_task = Task(
    description=(
        "Review the formatted insurance letter for compliance.\n"
        "Ensure:\n"
        "- Regulatory notice is present\n"
        "- Appeal rights are included when applicable\n"
        "- Policy and claim references are correct\n\n"
        "If something is missing, ADD it.\n"
        "Return the final compliant letter."
    ),
    expected_output="A fully compliant insurance letter",
    agent=compliance_agent,
)

In [14]:
from crewai import Crew

insurance_letter_crew = Crew(
    agents=[
        letter_generator_agent,
        formatter_agent,
        compliance_agent,
    ],
    tasks=[
        generate_letter_task,
        format_letter_task,
        compliance_task,
    ],
    verbose=True,
)

In [ ]:
inputs = {
    "letter_type": "Denial Letter",
    "company_name": "Cascade Assurance",
    "insured_name": "Ananya Brown",
    "policy_number": "P-4903497",
    "claim_number": "C-8627060",
}

final_letter = insurance_letter_crew.kickoff(inputs=inputs)

print("\n✅ FINAL INSURANCE LETTER\n")
print(final_letter)


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 4f865ff6-4eec-47e9-a966-122e4a74273e                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Insurance Letter Generator                                                                              │
│                                                                                                                 │
│  Task: Using the canonical instruction set below, generate an insurance letter.                                 │
│                                                                                                                 │
│  Canonical Instructions:                                                                                        │
│  CANONICAL INSTRUCTIONS FOR INSURANCE LETTER GENERATION                                                         │
│  ==========================================================================================                     │
│                                                                                                                 │
│  1. **Header Section**: Include the company name, address, and date at the top of the letter.                   │
│     - Example:                                                                                                  │
│       ```                                                                                                       │
│       [Company Name]                                                                                            │
│       [Company Address]                                                                                         │
│       [City, State, Zip Code]                                                                                   │
│       [Date]                                                                                                    │
│       ```                                                                                                       │
│                                                                                                                 │
│  2. **Subject Line**: Clearly state the purpose of the letter.                                                  │
│     - Example:                                                                                                  │
│       ```                                                                                                       │
│       Subject: [Type of Letter: Denial Letter / Coverage Decision / Request for Additional Information]         │
│       ```                                                                                                       │
│                                                                                                                 │
│  3. **Salutation**: Address the recipient by name.                                                              │
│     - Example:                                                                                                  │
│       ```                                                                                                       │
│       Dear [Recipient's Name],                                                                                  │
│       ```                                                                                                       │
│                                                                                                                 │
│  4. **Claim Information**: Reference the specific insurance policy and claim number.                            │
│     - Example:                                                                                                  │
│       ```                                                                                                       │
│       We are writing regarding your insurance policy [P

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Insurance Letter Generator                                                                              │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```                                                                                                            │
│  Cascade Assurance                                                                                              │
│  1234 Insurance Drive                                                                                           │
│  Anytown, State 12345                                                                                           │
│  October 21, 2023                                                                                               │
│                                                                                                                 │
│  Subject: Denial Letter                                                                                         │
│                                                                                                                 │
│  Dear Ananya Brown,                                                                                             │
│                                                                                                                 │
│  We are writing regarding your insurance policy P-4903497 and claim number C-8627060.                           │
│                                                                                                                 │
│  Claim Review Summary                                                                                           │
│  --------------------                                                                                           │
│  We carefully reviewed your claim submission and supporting documentation.                                      │
│                                                                                                                 │
│  Coverage Determination                                                                                         │
│  ----------------------                                                                                         │
│  After evaluation, we regret to inform you that coverage cannot be extended for this claim based on policy      │
│  exclusions and conditions.                                                                                     │
│                                                                                                                 │
│  Your Options / Next Steps                                                                                      │
│  --------------------------                                                                                     │
│  - You may request a written explanation of this decision.                                                      │
│  - You may submit additional documentation for reconsideration.                                                 │
│  - Appeal rights are outlined in your policy.                                                                   │
│                                                                                                                 │
│  Compliance and Regulatory Notice                                                                               │
│  This correspondence is issued in accordance with applicable insurance laws and regulations. All                │
│  determinations are based on the terms, conditions, exc

Output()

...

Output()

╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 845d8594-a6d5-4d0a-893c-3481adf7d79b                                                                     │
│  Agent: Insurance Compliance Reviewer                                                                           │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── Execution Traces ──────────────────────────────╮
│                                                                              │
│  🔍 Detailed execution traces are available!                                 │
│                                                                              │
│  View insights including:                                                    │
│    • Agent decision-making process                                           │
│    • Task execution flow and timing                                          │
│    • Tool usage details                                                      │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯


Would you like to view your execution traces? [y/N] (20s timeout): 

╭────────────────────────── Tracing Preference Saved ──────────────────────────╮
│                                                                              │
│  Info: Tracing has been disabled.                                            │
│                                                                              │
│  Your preference has been saved. Future Crew/Flow executions will not        │
│  collect traces.                                                             │
│                                                                              │
│  To enable tracing later, do any one of these:                               │
│  • Set tracing=True in your Crew/Flow code                                   │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file               │
│  • Run: crewai traces enable                                                 │
│                                       